In [1]:
# Import packages

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load the Excel files

min_wage_df = pd.read_excel('annual_wage_min.xlsx', engine='openpyxl') 
gdp_per_capita_df = pd.read_excel('gdp_per_capita.xlsx', engine='openpyxl')
quality_of_life_df = pd.read_excel('quality_of_life.xlsx', engine='openpyxl')
cost_df = pd.read_excel('cost_of_living_index.xlsx', engine='openpyxl')

# Source: 
# https://en.wikipedia.org/wiki/List_of_countries_by_minimum_wage
# https://www.worldometers.info/gdp/gdp-per-capita/
# https://www.worlddata.info/richest-countries.php
# https://www.numbeo.com/cost-of-living/rankings_by_country.jsp?title=2022

## File Cleaning

In [3]:
# Check "min_wage" file

min_wage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country          159 non-null    object 
 1   Annual min wage  158 non-null    float64
 2   Work hours/week  159 non-null    float64
dtypes: float64(2), object(1)
memory usage: 3.9+ KB


In [4]:
# need to convert "work hours/week" from object to float

min_wage_df["Work hours/week"] = min_wage_df["Work hours/week"].astype(float)

In [5]:
# Verify convertion to float

min_wage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country          159 non-null    object 
 1   Annual min wage  158 non-null    float64
 2   Work hours/week  159 non-null    float64
dtypes: float64(2), object(1)
memory usage: 3.9+ KB


In [6]:
# Check "gdp_per_capita" file

gdp_per_capita_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Country         176 non-null    object
 1   GDP per capita  176 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.9+ KB


In [7]:
# Check "quality of life" file

quality_of_life_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Country     129 non-null    object
 1   Stability   129 non-null    int64 
 2   Rights      129 non-null    int64 
 3   Health      129 non-null    int64 
 4   Safety      129 non-null    int64 
 5   Climate     129 non-null    int64 
 6   Costs       129 non-null    int64 
 7   Popularity  129 non-null    int64 
dtypes: int64(7), object(1)
memory usage: 8.2+ KB


In [8]:
# Check "cost" file

cost_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Country               139 non-null    object 
 1   Cost of Living Index  139 non-null    float64
dtypes: float64(1), object(1)
memory usage: 2.3+ KB


In [9]:
# Merge dataframes based on the 'Country' column

merged_df = min_wage_df.merge(gdp_per_capita_df, on='Country', how='outer')
merged_df = merged_df.merge(quality_of_life_df, on='Country', how='outer')
merged_df = merged_df.merge(cost_df, on='Country', how='outer')

merged_df

,Country,Annual min wage,Work hours/week,GDP per capita,Stability,Rights,Health,Safety,Climate,Costs,Popularity,Cost of Living Index
0,Afghanistan,858.0,40.0,NaN,30.0,9.0,15.0,56.0,64.0,63.0,12.0,20.4
1,Albania,4637.0,40.0,6643.0,57.0,43.0,60.0,86.0,55.0,59.0,19.0,35.5
2,Algeria,1777.0,40.0,4274.0,40.0,24.0,59.0,80.0,76.0,54.0,31.0,26.9
3,Andorra,18253.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,663.0,44.0,2999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
206,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.1
207,Venezuela,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.2
208,Bosnia And Herzegovina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.1
209,Somalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.6


In [10]:
#Remove rows with "NaN" values

cleaned_df = merged_df.dropna()
cleaned_df
#90 countries worth of data

,Country,Annual min wage,Work hours/week,GDP per capita,Stability,Rights,Health,Safety,Climate,Costs,Popularity,Cost of Living Index
1,Albania,4637.0,40.0,6643.0,57.0,43.0,60.0,86.0,55.0,59.0,19.0,35.5
2,Algeria,1777.0,40.0,4274.0,40.0,24.0,59.0,80.0,76.0,54.0,31.0,26.9
6,Argentina,16149.0,48.0,13904.0,44.0,42.0,77.0,81.0,73.0,42.0,38.0,34.7
7,Armenia,1787.0,40.0,7014.0,43.0,50.0,64.0,76.0,50.0,54.0,19.0,33.9
8,Australia,34515.0,38.0,64003.0,87.0,95.0,87.0,94.0,90.0,23.0,55.0,77.7
...,...,...,...,...,...,...,...,...,...,...,...,...
153,United States,15080.0,40.0,75269.0,67.0,83.0,69.0,67.0,66.0,33.0,87.0,70.1
154,Uruguay,5816.0,48.0,20795.0,75.0,82.0,70.0,80.0,78.0,42.0,39.0,52.1
155,Uzbekistan,1041.0,40.0,2322.0,51.0,23.0,59.0,81.0,53.0,61.0,36.0,27.3
157,Vietnam,1591.0,40.0,4164.0,71.0,35.0,51.0,80.0,52.0,50.0,38.0,37.5


In [11]:
singapore_df = merged_df[merged_df['Country'] == 'Singapore']

singapore_df

,Country,Annual min wage,Work hours/week,GDP per capita,Stability,Rights,Health,Safety,Climate,Costs,Popularity,Cost of Living Index
129,Singapore,NaN,44.0,78115.0,82.0,89.0,79.0,100.0,45.0,53.0,51.0,84.0


## Building regression model

In [12]:
## fitting the data into a model

x_columns = ["Work hours/week", "GDP per capita", "Stability",  "Rights", "Health", "Safety", "Climate", "Costs", "Popularity", "Cost of Living Index"]
X = cleaned_df[x_columns]
y = cleaned_df["Annual min wage"]
linear_model = LinearRegression()
linear_model.fit(X, y)

LinearRegression()

In [13]:
# Split the data into training and testing sets (80% train, 20% test)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [14]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
# Initialize and train the linear regression model

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [17]:
# Predict the minimum income for Singapore
singapore_features = singapore_df[x_columns] 
singapore_features = scaler.transform(singapore_features)  # Standardize the features
singapore_pred = model.predict(singapore_features)

print(f"Estimated minimum income for Singapore: ${singapore_pred[0]:.2f}")

Estimated minimum income for Singapore: $27109.85


## Stepwise regression

In [21]:
#Define formula

def get_stats():
    X = cleaned_df[x_columns]
    results = sm.OLS(y, X).fit()
    print(results.summary())
get_stats()

                                 OLS Regression Results                                
Dep. Variable:        Annual min wage   R-squared (uncentered):                   0.893
Model:                            OLS   Adj. R-squared (uncentered):              0.879
Method:                 Least Squares   F-statistic:                              66.69
Date:                Mon, 25 Sep 2023   Prob (F-statistic):                    1.36e-34
Time:                        11:51:51   Log-Likelihood:                         -873.49
No. Observations:                  90   AIC:                                      1767.
Df Residuals:                      80   BIC:                                      1792.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

## Remove variables that are most statistically significant (highest p-value)

In [22]:
x_columns.remove("Work hours/week")

get_stats()

                                 OLS Regression Results                                
Dep. Variable:        Annual min wage   R-squared (uncentered):                   0.893
Model:                            OLS   Adj. R-squared (uncentered):              0.881
Method:                 Least Squares   F-statistic:                              74.95
Date:                Mon, 25 Sep 2023   Prob (F-statistic):                    1.58e-35
Time:                        11:51:51   Log-Likelihood:                         -873.53
No. Observations:                  90   AIC:                                      1765.
Df Residuals:                      81   BIC:                                      1788.
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

In [23]:
x_columns.remove("Climate")

get_stats()

                                 OLS Regression Results                                
Dep. Variable:        Annual min wage   R-squared (uncentered):                   0.892
Model:                            OLS   Adj. R-squared (uncentered):              0.881
Method:                 Least Squares   F-statistic:                              84.67
Date:                Mon, 25 Sep 2023   Prob (F-statistic):                    2.21e-36
Time:                        11:51:51   Log-Likelihood:                         -873.85
No. Observations:                  90   AIC:                                      1764.
Df Residuals:                      82   BIC:                                      1784.
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

In [24]:
x_columns.remove("Popularity")

get_stats()

                                 OLS Regression Results                                
Dep. Variable:        Annual min wage   R-squared (uncentered):                   0.891
Model:                            OLS   Adj. R-squared (uncentered):              0.882
Method:                 Least Squares   F-statistic:                              97.24
Date:                Mon, 25 Sep 2023   Prob (F-statistic):                    2.80e-37
Time:                        11:51:51   Log-Likelihood:                         -874.15
No. Observations:                  90   AIC:                                      1762.
Df Residuals:                      83   BIC:                                      1780.
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

In [25]:
x_columns.remove("Rights")

get_stats()

                                 OLS Regression Results                                
Dep. Variable:        Annual min wage   R-squared (uncentered):                   0.890
Model:                            OLS   Adj. R-squared (uncentered):              0.882
Method:                 Least Squares   F-statistic:                              113.3
Date:                Mon, 25 Sep 2023   Prob (F-statistic):                    4.05e-38
Time:                        11:51:51   Log-Likelihood:                         -874.67
No. Observations:                  90   AIC:                                      1761.
Df Residuals:                      84   BIC:                                      1776.
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

# Building regression with statistically significant variables

In [29]:
## creating a linear model and prediction

print(x_columns)
X = cleaned_df[x_columns]
y = cleaned_df["Annual min wage"]
linear_model = LinearRegression()
linear_model.fit(X, y)


# singapore_features = singapore_df[x_columns] 
# singapore_features = scaler.transform(singapore_features)
# y_pred = linear_model.predict(singapore_features)
# print("Prediction for Singapore is ", y_pred)

['GDP per capita', 'Stability', 'Health', 'Safety', 'Costs', 'Cost of Living Index']


LinearRegression()

In [30]:
# Split the data into training and testing sets (80% train, 20% test)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [31]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [32]:
# Initialize and train the linear regression model

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [34]:
# Predict the minimum income for Singapore
singapore_features = singapore_df[x_columns] 
singapore_features = scaler.transform(singapore_features)  # Standardize the features
singapore_pred = model.predict(singapore_features)

print(f"Estimated minimum income for Singapore: ${singapore_pred[0]:.2f}")

Estimated minimum income for Singapore: $27848.98
